In [1]:
pip install torch==1.11.0+cu113 torchvision==0.12.0+cu113 torchaudio==0.11.0 torchtext==0.12.0 --extra-index-url https://download.pytorch.org/whl/cu113

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cu113
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 1.6/1.6 GB 90.7 MB/s eta 0:00:01tcmalloc: large alloc 1636999168 bytes == 0x34ac000 @  0x7f8a8c822680 0x7f8a8c843824 0x5b3128 0x5bbc90 0x5f714c 0x64d800 0x527022 0x504866 0x56bbe1 0x569d8a 0x5f60c3 0x56bbe1 0x569d8a 0x5f60c3 0x56bbe1 0x569d8a 0x5f60c3 0x56bbe1 0x569d8a 0x5f60c3 0x56bbe1 0x569d8a 0x5f60c3 0x56bbe1 0x5f5ee6 0x56bbe1 0x569d8a 0x5f60c3 0x56cc92 0x569d8a 0x5f60c3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 1.6/1.6 GB 91.5 MB/s eta 0:00:01tcmalloc: large alloc 2046255104 bytes == 0x64dd6000 @  0x7f8a8c822680 0x7f8a8c842da2 0x5f714c 0x64d800 0x527022 0x504866 0x56bbe1 0x569d8a 0x5f60c3 0x56bbe1 0x569d8a 0x5f60c3 0x56bbe1 0x569d8a 0x5f60c3 0x56bbe1 0x569d8a 0x5f60c3 0x56bbe1 0x569d8a 0x5f60c3 0x56bbe1 0x5f5ee6 0x56bbe1 0x569d8a 0x5f60c3 0x56cc92 0x569d8a 0x5f60c3 0x56bbe1 0x569d8a
     ━━━━━━━

In [2]:
import scipy
import torch
import matplotlib.pyplot as plt

In [3]:
# To install Falkon library
!pip install git+https://github.com/falkonml/falkon.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/falkonml/falkon.git to /tmp/pip-req-build-omypfipd
  Running command git clone --filter=blob:none --quiet https://github.com/falkonml/falkon.git /tmp/pip-req-build-omypfipd
  Resolved https://github.com/falkonml/falkon.git to commit e0f35851b00e181d6f2a5c52fc1aa160b1e7b0d6
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/getkeops/keops (to revision ad044a671fdc3c2790b0321f6b9f9b5aa3d220df) to /tmp/pip-install-ih75r3of/keopscore_b6a126d7422d416e82219bb18b79784a
  Running command git clone --filter=blob:none --quiet https://github.com/getkeops/keops /tmp/pip-install-ih75r3of/keopscore_b6a126d7422d416e82219bb18b79784a
  Running command git rev-parse -q --verify 'sha^ad044a671fdc3c2790b0321f6b9f9b5aa3d220df'
  Running command git fetch -q https://github.com/getkeops/keops ad044a671fdc3c2790b0321f6b9f9b5aa3d220df
  Running command git checkout 

In [4]:
import falkon

[KeOps] Warning : cuda was detected, but driver API could not be initialized. Switching to cpu only.


In [5]:
# Drive
from google.colab import drive

# we connect to the drive folder to work with the data stored
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import numpy as np
import pandas as pd

In [7]:
import os, time

In [8]:
import numpy as np

import os, time

import torch

from falkon import LogisticFalkon
from falkon.kernels import GaussianKernel
from falkon.options import FalkonOptions
from falkon.gsc_losses import WeightedCrossEntropyLoss

import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager

from sklearn import metrics
from scipy.spatial.distance import pdist
from scipy.stats import norm, chi2, rv_continuous, kstest


# UTILS

def candidate_sigma(data, perc=90):
    # this function estimates the width of the gaussian kernel.
    # use on a (small) sample of reference data (standardize first if necessary)
    pairw = pdist(data)
    return round(np.percentile(pairw,perc),1)

'''
def NP2_gen(size, seed):
    # custom function to generate samples of non-resonant new physics events
    if size>10000:
        raise Warning('Sample size is grater than 1000: Generator will not approximate the tail well')
    sample = np.array([])
    #normalization factor                                                                                                                                    
    np.random.seed(seed)
    Norm = 256.*0.25*0.25*np.exp(-2)
    while(len(sample)<size):
        x = np.random.uniform(0,1) #assuming not to generate more than 10 000 events                                                                         
        p = np.random.uniform(0, Norm)
        if p<= 256.*x*x*np.exp(-8.*x):
            sample = np.append(sample, x)
    return sample
'''

class non_res(rv_continuous):

    def _pdf(self, x):

        return 256 * (x**2) * np.exp(- 8 * x)

def nonres_sig(N_S, seed):
    # this function can be used to generate non-resonant signal events.
    
    my_sig = non_res(momtype = 0, a=0, b=1, seed=seed)
    
    sig_sample = my_sig.rvs(size = N_S)
    
    return sig_sample


def get_logflk_config(M,flk_sigma,lam,weight,iter=[100],seed=None,cpu=False):
    # it returns logfalkon parameters
    return {
            'kernel' : GaussianKernel(sigma=flk_sigma),
            'M' : M, #number of Nystrom centers,
            'penalty_list' : lam, # list of regularization parameters,
            'iter_list' : iter, #list of number of CG iterations,
            'options' : FalkonOptions(cg_tolerance=np.sqrt(1e-7), keops_active='no', use_cpu=cpu, debug = False),
            'seed' : seed, # (int or None), the model seed (used for Nystrom center selection) is manually set,
            'loss' : WeightedCrossEntropyLoss(kernel=GaussianKernel(sigma=flk_sigma), neg_weight=weight),
            }


def compute_t(preds,Y,weight):
    # it returns extended log likelihood ratio from predictions
    diff = weight*np.sum(1 - np.exp(preds[Y==0]))
    return 2 * (diff + np.sum(preds[Y==1]))

def trainer(X,Y,flk_config):
    # trainer for logfalkon model
    Xtorch=torch.from_numpy(X)
    Ytorch=torch.from_numpy(Y)
    model = LogisticFalkon(**flk_config)
    model.fit(Xtorch, Ytorch)
    return model.predict(Xtorch).numpy()

def standardize(X):
    # standardize data as in HIGGS and SUSY
    for j in range(X.shape[1]):
        column = X[:, j]

        mean = np.mean(column)
        std = np.std(column)
    
        if np.min(column) < 0:
            column = (column-mean)*1./ std
        elif np.max(column) > 1.0:                                                                                                                                        
            column = column *1./ mean
    
        X[:, j] = column
    
    return X

def return_best_chi2dof(tobs):
    """
    Returns the most fitting value for dof assuming tobs follows a chi2_dof distribution,
    computed with a Kolmogorov-Smirnov test, removing NANs and negative values.
    Parameters
    ----------
    tobs : np.ndarray
        observations
    Returns
    -------
        best : tuple
            tuple with best dof and corresponding chi2 test result
    """
    
    
    dof_range = np.arange(np.nanmedian(tobs) - 10, np.nanmedian(tobs) + 10, 0.1)
    
    ks_tests = []
    
    for dof in dof_range:
        
        test = kstest(tobs, lambda x:chi2.cdf(x, df=dof))[0]
        
        ks_tests.append((dof, test))
        
    ks_tests = [test for test in ks_tests if test[1] != 'nan'] # remove nans
    
    ks_tests = [test for test in ks_tests if test[0] >= 0] # retain only positive dof
        
    best = min(ks_tests, key = lambda t: t[1]) # select best dof according to KS test result
        
    return best

In [9]:
def emp_zscore(t0,t1):
    if max(t0) <= t1:
        p_obs = 1 / len(t0)
        Z_obs = round(norm.ppf(1 - p_obs),2)
        return Z_obs
    else:
        p_obs = np.count_nonzero(t0 >= t1) / len(t0)
        Z_obs = round(norm.ppf(1 - p_obs),2)
        return Z_obs

def chi2_zscore(t1, dof):
    p = chi2.cdf(float('inf'),dof)-chi2.cdf(t1,dof)
    return norm.ppf(1 - p)

In [10]:
# for 2-D without plot-reconstruction (with passing data)

def run_toys(sig, data, fraud, genuine, output_path, N_0, N0, NS, flk_config, toys=np.arange(100), plots_freq=0, df=10, savefig=True):

    '''
    type of signal: "NP0", "NP1", "NP2", "NP3"
    output_path: directory (inside ./runs/) where to save results
    N_0: size of ref sample
    N0: expected num of bkg events
    NS: expected num of signal events
    flk_config: dictionary of logfalkon parameters
    toys: numpy array with seeds for toy generation
    plots_freq: how often to plot inputs with learned reconstructions
    df: degree of freedom of chi^2 for plots
    '''

    output_path = "./runs/" + output_path
    os.makedirs(output_path, exist_ok=True)

    #save config file (temporary solution)
    with open(output_path+"/flk_config.txt","w") as f:
        f.write( str(flk_config) )

    weight = N0/N_0
    # passing data
    samples = data

    
    # dividing the dataset
    fraud_df = fraud
    non_fraud_df = genuine

    dim = 2

    for i in toys:

        st_time = time.time()

        rng = np.random.default_rng(i)

        # Shuffling the data
        samples = samples.sample(frac=1, random_state=i)
        fraud_df = fraud_df.sample(frac=1,random_state=i)
        non_fraud_df = non_fraud_df.sample(frac=1,random_state=i)

        N0p = rng.poisson(lam=N0)
        if sig!="NP0": NSp = rng.poisson(lam=NS) # if data contains anomalies
        else: NSp = 0

        N = N_0 + N0p + NSp

        if sig=="NP0":
            X = non_fraud_df[['V11','V14']].sample(n= N, replace=False, random_state=i) # both reference and data contain only bkg events (no NP component)
        else:
            X = np.zeros(shape=(N,dim))
            X[:N_0+N0p,:] = non_fraud_df[['V11','V14']].sample(n= N_0+N0p, replace=False, random_state=i) # ref and bkg
            X[N_0+N0p:,:] = fraud_df[['V11','V14']].sample(n= NSp, replace=False, random_state=i) # signal
        # initialize labes
        Y = np.zeros(shape=(N,1))
        Y[N_0:,:] = np.ones((N0p+NSp,1)) # flip data labels to one
        X = np.array(X)
        
        # learn_t
        flk_config['seed']=i # select different centers for different toys

        preds = trainer(X,Y,flk_config)

        t = compute_t(preds,Y,weight)

        dt = round(time.time()-st_time,2)

        
        with open(output_path+"t.txt", 'a') as f:
            f.write('{},{}\n'.format(i,t))

In [11]:
def get_results(ref, data, dof=None):
  md_tref = round(np.median(ref), 2)
  md_tdata = round(np.median(data), 2)
  max_zemp = emp_zscore(ref,np.max(ref))
  zemp = emp_zscore(ref,np.median(data))
  if dof:
      z_chi2 = round(chi2_zscore(np.median(data),dof=dof),2)

  return {"md_tref":md_tref,
          "md_tdata":md_tdata,
          "max_zemp":max_zemp,
          "zemp":zemp,
          "dof":dof,
          "z_chi2":z_chi2}

In [22]:
import itertools

def GridSearch(parms):

  res = []

  # Loading the dataset
  creds = pd.read_csv("/content/drive/MyDrive/CreditCards/creditcard.csv")
  fraud_samples = creds.loc[creds['Class'] == 1]
  genuine_samples = creds.loc[creds['Class'] == 0]

  combinations = itertools.product(*(parms[Name] for Name in parms.keys()))
  for row in list(combinations):

    # M should be less than N_0 and N0
    if row[2]<row[0] and row[2]<row[1]:
      # parameters
      flk_sigma = 3
      lam = 1e-6
      M = 100
      N_0, N0, NS = row[0], row[1], row[2]
      weight = N0/N_0

      flk_config = get_logflk_config(M,flk_sigma,[lam],weight=weight,iter=[100],seed=None,cpu=True)

      # Running the codes on reference dataset 
      NS_temp = 0
      run_toys("NP0", creds, fraud_samples, genuine_samples, "./reference/", N_0, N0, NS_temp, flk_config, toys=np.arange(100), df= 22)

      # Best DoF
      best = return_best_chi2dof(np.loadtxt('/content/runs/reference/t.txt',delimiter=',')[:,1])

      # Running the codes on all data
      run_toys("NP1", creds, fraud_samples, genuine_samples, "./NP1/", N_0, N0, NS, flk_config, toys=np.arange(100), df= best[0])

      # Collecting the important results for comparison
      results = get_results(np.loadtxt('/content/runs/reference/t.txt',delimiter=',')[:,1],np.loadtxt('/content/runs/NP1/t.txt',delimiter=',')[:,1],dof=best[0])

      # adding the parameters info to the collected data for comparison
      results.update({"N_0": N_0, "N0": N0, "NS": NS})
      # Updating the whole Results
      res.append(results)

      #clearing previous results
      !rm /content/runs/reference/t.txt
      !rm /content/runs/NP1/t.txt


  return res

In [23]:
parameters = {"N_0":[1000,3000],"N0":[600,800],"NS":[5,10,15,20,50]}
my_dict = GridSearch(parameters)
res_df = pd.DataFrame(my_dict)

Iteration 0 - penalty 1.000000e-06 - sub-iterations 100
Iteration 0 - penalty 1.000000e-06 - sub-iterations 100
Iteration 0 - penalty 1.000000e-06 - sub-iterations 100
Iteration 0 - penalty 1.000000e-06 - sub-iterations 100
Iteration 0 - penalty 1.000000e-06 - sub-iterations 100
Iteration 0 - penalty 1.000000e-06 - sub-iterations 100
Iteration 0 - penalty 1.000000e-06 - sub-iterations 100
Iteration 0 - penalty 1.000000e-06 - sub-iterations 100
Iteration 0 - penalty 1.000000e-06 - sub-iterations 100
Iteration 0 - penalty 1.000000e-06 - sub-iterations 100
Iteration 0 - penalty 1.000000e-06 - sub-iterations 100
Iteration 0 - penalty 1.000000e-06 - sub-iterations 100
Iteration 0 - penalty 1.000000e-06 - sub-iterations 100
Iteration 0 - penalty 1.000000e-06 - sub-iterations 100
Iteration 0 - penalty 1.000000e-06 - sub-iterations 100
Iteration 0 - penalty 1.000000e-06 - sub-iterations 100
Iteration 0 - penalty 1.000000e-06 - sub-iterations 100
Iteration 0 - penalty 1.000000e-06 - sub-iterati

In [24]:
res_df

,md_tref,md_tdata,max_zemp,zemp,dof,z_chi2,N_0,N0,NS
0,33.88,43.68,2.33,0.99,33.878090,1.17,1000,600,5
1,33.88,54.51,2.33,2.33,33.878090,2.20,1000,600,10
2,33.88,64.55,2.33,2.33,33.878090,3.05,1000,600,15
3,33.88,76.85,2.33,2.33,33.878090,3.98,1000,600,20
4,33.88,167.49,2.33,2.33,33.878090,inf,1000,600,50
5,40.32,48.81,2.33,0.95,40.522926,0.94,1000,800,5
6,40.32,57.50,2.33,1.75,40.522926,1.74,1000,800,10
7,40.32,66.81,2.33,2.33,40.522926,2.53,1000,800,15
8,40.32,82.34,2.33,2.33,40.522926,3.69,1000,800,20
9,40.32,166.58,2.33,2.33,40.522926,inf,1000,800,50


In [25]:
res_df.to_csv("GridSearch_Results.csv",index=False)

In [26]:
res_df.sort_values(by='z_chi2')

,md_tref,md_tdata,max_zemp,zemp,dof,z_chi2,N_0,N0,NS
15,24.50,32.14,2.33,0.99,25.795383,0.91,3000,800,5
5,40.32,48.81,2.33,0.95,40.522926,0.94,1000,800,5
10,23.05,29.83,2.33,1.08,23.452265,0.95,3000,600,5
0,33.88,43.68,2.33,0.99,33.878090,1.17,1000,600,5
6,40.32,57.50,2.33,1.75,40.522926,1.74,1000,800,10
16,24.50,42.90,2.33,2.33,25.795383,2.08,3000,800,10
1,33.88,54.51,2.33,2.33,33.878090,2.20,1000,600,10
11,23.05,41.21,2.33,2.05,23.452265,2.22,3000,600,10
7,40.32,66.81,2.33,2.33,40.522926,2.53,1000,800,15
17,24.50,51.60,2.33,2.33,25.795383,2.90,3000,800,15


In [27]:
res_df.groupby(['N_0','N0','NS']).mean()

md_tref  md_tdata  max_zemp  zemp        dof  z_chi2
N_0  N0  NS                                                      
1000 600 5     33.88     43.68      2.33  0.99  33.878090    1.17
         10    33.88     54.51      2.33  2.33  33.878090    2.20
         15    33.88     64.55      2.33  2.33  33.878090    3.05
         20    33.88     76.85      2.33  2.33  33.878090    3.98
         50    33.88    167.49      2.33  2.33  33.878090     inf
     800 5     40.32     48.81      2.33  0.95  40.522926    0.94
         10    40.32     57.50      2.33  1.75  40.522926    1.74
         15    40.32     66.81      2.33  2.33  40.522926    2.53
         20    40.32     82.34      2.33  2.33  40.522926    3.69
         50    40.32    166.58      2.33  2.33  40.522926     inf
3000 600 5     23.05     29.83      2.33  1.08  23.452265    0.95
         10    23.05     41.21      2.33  2.05  23.452265    2.22
         15    23.05     49.16      2.33  2.33  23.452265    2.98
         20    23.05     62.91      2.33  2.33  23.452265    4.13
         50    23.05    139.19      2.33  2.33  23.452265     inf
     800 5     24.50     32.14      2.33  0.99  25.795383    0.91
         10    24.50     42.90      2.33  2.33  25.795383    2.08
         15    24.50     51.60      2.33  2.33  25.795383    2.90
         20    24.50     61.35      2.33  2.33  25.795383    3.72
         50    24.50    144.31      2.33  2.33  25.795383     inf

In [28]:
import plotly.express as px

fig = px.line(res_df[res_df['N_0']==3000], x='NS', y='z_chi2', color = 'N0', text='z_chi2')
fig.update_layout(title = { 'text' : "Z_Chi^2--vs--NS\n (N_0= 3000, M= 100, lam= 1e-6)", 'x' : 0.5 })
fig.update_traces(textposition="middle center")
fig.show()

In [29]:
fig = px.line(res_df[res_df['N_0']==1000], x='NS', y='z_chi2', color = 'N0', text='z_chi2')
fig.update_layout(title = { 'text' : "Z_Chi^2--vs--NS\n (N_0= 1000, M= 100, lam= 1e-6)", 'x' : 0.5 })
fig.update_traces(textposition="middle center")
fig.show()

In [30]:
fig = px.line(res_df[res_df['N0']==600], x='NS', y='z_chi2', color = 'N_0', text='z_chi2')
fig.update_layout(title = { 'text' : "Z_Chi^2--vs--NS\n (N0= 600, M= 100, lam= 1e-6)", 'x' : 0.5 })
fig.update_traces(textposition="middle center")
fig.show()

In [31]:
fig = px.line(res_df[res_df['N0']==800], x='NS', y='z_chi2', color = 'N_0', text='z_chi2')
fig.update_layout(title = { 'text' : "Z_Chi^2--vs--NS\n (N0= 800, M= 100, lam= 1e-6)", 'x' : 0.5 })
fig.update_traces(textposition="middle center")
fig.show()